In [1]:
from completed_missing_data import completed_missing_data, convert_historical_date_to_date_difference, remove_described_col, map_target
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:


train_org = pd.read_csv('data/train.csv')
train = completed_missing_data(train_org)
train = convert_historical_date_to_date_difference(train)
train = map_target(train)

#train_with_league_id = pd.concat([train, pd.get_dummies(train['league_id'],prefix='league_')], axis=1)

# train = remove_described_col(train)
# train_with_league_id = remove_described_col(train_with_league_id)



/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove 9 and 10 day historii
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


In [2]:
test_org = pd.read_csv('data/test.csv')
test = completed_missing_data(test_org)
test = convert_historical_date_to_date_difference(test)
test = remove_described_col(test)
test

correction col type..
percent of object with nan value:  39.35
remove coach columns..
percent of object with nan value:  13.93
remove object with more that  col 80 empty...
percent of object with nan value:  13.93
remove 9 and 10 day historii
percent of object with nan value:  11.15
remove all nan data
percent of object with nan value:  11.15


,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_is_play_home_1,...,away_team_history_rating_7,away_team_history_rating_8,away_team_history_opponent_rating_1,away_team_history_opponent_rating_2,away_team_history_opponent_rating_3,away_team_history_opponent_rating_4,away_team_history_opponent_rating_5,away_team_history_opponent_rating_6,away_team_history_opponent_rating_7,away_team_history_opponent_rating_8
0,0,71,97,70,96,143,98,120,288,0.0,...,4.972890,3.150812,4.721590,6.953965,6.535228,5.782611,7.538950,8.428800,4.922229,6.796484
1,0,166,169,168,143,360,94,193,72,0.0,...,6.765418,6.203948,4.459100,4.543053,4.861583,5.082300,9.011000,6.621075,4.141575,5.056690
3,0,48,98,46,48,918,72,97,95,0.0,...,8.730013,7.177425,3.790810,3.562960,4.655450,4.232500,3.999711,4.095675,3.746925,4.952475
4,0,48,98,48,48,917,68,100,95,0.0,...,6.550694,3.018285,6.000775,7.648230,5.580242,6.971550,5.624680,5.733725,4.453211,8.787767
5,0,48,98,46,50,893,96,96,119,0.0,...,3.746925,4.472350,5.304272,7.069420,6.717443,6.130100,5.281494,5.731775,8.730013,6.617900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72706,0,145,163,169,190,122,216,142,219,1.0,...,7.010503,6.770769,5.418217,4.389327,4.568866,6.076045,3.720895,4.656597,4.494093,3.665037
72707,0,141,98,286,120,96,144,95,97,0.0,...,4.458606,5.017476,4.839404,5.167733,5.403118,5.803769,5.873700,5.143589,6.695206,6.109570
72708,0,913,765,1875,93,450,1452,77,1097,1.0,...,12.404786,5.000000,4.828760,4.448000,8.047350,7.526000,7.597350,6.304886,2.612821,5.000000
72709,0,75,92,71,72,77,69,71,70,0.0,...,5.390842,4.837319,4.963700,5.099264,6.111216,3.620043,6.283775,3.359233,4.382062,5.823854


# Dodanie cech
- srednia ilosc goli  na mecz w lidzie 
- sredni ranking drużyn w lidzie
- sredni ilosc meczow wygranych w lidze



In [3]:

# home_mean_ratting = train[last_match_columns].groupby(by='home_team_history_league_id_1').mean().reset_index().sort_values(by='home_team_history_league_id_1', ascending = False)
home_mean_ratting_all =pd.DataFrame()
for i in range(1,9):
    for home_or_away in ['home', 'away']:
        last_match_columns  =  [x for x in train.columns  if  str(i) in x]
        rating_col_name = home_or_away + '_team_history_league_id_'+str(i)
        home_mean_ratting = train[last_match_columns].groupby(by=rating_col_name).mean()
        # train['league_name']
        home_mean_ratting['mean_ratting_'+home_or_away +'_' + str(i)] = (home_mean_ratting[home_or_away +'_team_history_rating_'+str(i)] + home_mean_ratting[home_or_away+'_team_history_opponent_rating_'+str(i)]) / 2
        home_mean_ratting_all = home_mean_ratting_all.append(home_mean_ratting[ 'mean_ratting_'+home_or_away+ '_'+str(i)])

home_mean_ratting_all =home_mean_ratting_all.mean().reset_index().rename(columns = {'index': 'league_id',0: 'league_id_ratting'})
home_mean_ratting_all

,league_id,league_id_ratting
0,2.0,6.805677
1,5.0,6.524094
2,8.0,6.374938
3,9.0,5.717973
4,12.0,5.817057
...,...,...
830,782.0,9.903354
831,977.0,4.259858
832,1194.0,5.092400
833,1629.0,5.653429


In [4]:
train = train.merge(home_mean_ratting_all,on ='league_id').sort_values(by='league_id')
train = remove_described_col(train)
# %%
train

,target,league_id,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,...,away_team_history_opponent_rating_8,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,league_id_ratting
61326,2,2,1,99,71,406,98,99,69,98,...,6.986060,337.0,2.0,334.0,334.0,2.0,2.0,334.0,1101.0,6.805677
61329,2,2,1,144,75,96,93,314,93,98,...,4.315007,325.0,2.0,325.0,331.0,5.0,325.0,325.0,325.0,6.805677
61328,2,2,1,144,104,66,119,290,93,169,...,3.938007,444.0,2.0,444.0,2.0,444.0,2.0,444.0,2.0,6.805677
61327,1,2,1,80,64,98,95,97,406,100,...,6.193325,208.0,2.0,208.0,2.0,208.0,1101.0,208.0,208.0,6.805677
61325,3,2,1,98,94,95,196,91,192,169,...,4.545117,609.0,2.0,609.0,1545.0,609.0,609.0,609.0,609.0,6.805677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93939,2,2037,1,547,1344,463,165,211,168,464,...,5.657750,1739.0,80.0,1419.0,1419.0,80.0,80.0,80.0,80.0,7.900250
93938,3,2037,1,545,1133,715,168,168,168,505,...,5.092750,1739.0,80.0,1419.0,1419.0,80.0,80.0,80.0,80.0,7.900250
93937,1,2037,1,504,1344,504,168,168,168,505,...,9.034953,1739.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,7.900250
93936,1,2037,1,504,1135,712,168,168,168,505,...,15.963227,1739.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,7.900250


In [5]:
for columns in train.columns:
    print(columns, train[columns].dtype)

target int64
league_id int64
is_cup int64
home_team_history_match_date_1 int64
home_team_history_match_date_2 int64
home_team_history_match_date_3 int64
home_team_history_match_date_4 int64
home_team_history_match_date_5 int64
home_team_history_match_date_6 int64
home_team_history_match_date_7 int64
home_team_history_match_date_8 int64
home_team_history_is_play_home_1 float64
home_team_history_is_play_home_2 float64
home_team_history_is_play_home_3 float64
home_team_history_is_play_home_4 float64
home_team_history_is_play_home_5 float64
home_team_history_is_play_home_6 float64
home_team_history_is_play_home_7 float64
home_team_history_is_play_home_8 float64
home_team_history_is_cup_1 float64
home_team_history_is_cup_2 float64
home_team_history_is_cup_3 float64
home_team_history_is_cup_4 float64
home_team_history_is_cup_5 float64
home_team_history_is_cup_6 float64
home_team_history_is_cup_7 float64
home_team_history_is_cup_8 float64
home_team_history_goal_1 float64
home_team_history_goa

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

for data, name in  zip([train],['train'] ):
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                        random_state=42)
    pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
    print(name, results)
# drzewach, svm
# kolumny
# kaggle
#

accuracy: 0.49
train [3 1 1 ... 1 1 3]


# Dobór cech

In [28]:


X_train, X_test, y_train, y_test = train_test_split(train.iloc[:, 1:], train.iloc[:, 0], test_size=0.33,
                                                    random_state=42)
model = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

grouped_columns = []
last_value = None
for column in X_train.columns:
    if grouped_columns and column[:-3] in last_value:
        grouped_columns[-1].append(column)
    else:
        grouped_columns.append([column])
    last_value = column



def  features_selection(grouped_columns):
    the_best_features = []
    the_best_score = 0
    for L in range(0, 1 + len(grouped_columns)):
        errors = []
        for value in grouped_columns:

            if value in the_best_features:
                continue
            elif type(value) == list:
                if value[0] in the_best_features:
                    continue

            tmp_the_best_feuteures = the_best_features.copy()

            if type(value) == list:
                tmp_the_best_feuteures.extend(value)  # dla weekday, mnths, seazon
            else:
                tmp_the_best_feuteures.append(value)

            error = np.mean(cross_val_score(model, X_train[tmp_the_best_feuteures], y_train))
            errors.append((tmp_the_best_feuteures, error))

        errors = [(k, v) for k, v in sorted(errors, key=lambda item: item[1])]

        if errors[-1][
            1] < the_best_score and the_best_score != 0:  # Jesli  dodanie cechy zwieksza bład, to zakoncz algorytm
            break

        the_best_features = errors[-1][0]
        the_best_score = errors[-1][1]

    print('the_best_feuteures\n', the_best_features)
    print('the_best_cross_value score\n', the_best_score)
    return  the_best_features, the_best_score
the_best_features, the_best_score = features_selection(grouped_columns)
the_best_features

the_best_feuteures
 ['away_team_history_rating_1', 'away_team_history_rating_2', 'away_team_history_rating_3', 'away_team_history_rating_4', 'away_team_history_rating_5', 'away_team_history_rating_6', 'away_team_history_rating_7', 'away_team_history_rating_8', 'home_team_history_rating_1', 'home_team_history_rating_2', 'home_team_history_rating_3', 'home_team_history_rating_4', 'home_team_history_rating_5', 'home_team_history_rating_6', 'home_team_history_rating_7', 'home_team_history_rating_8', 'home_team_history_opponent_goal_1', 'home_team_history_opponent_goal_2', 'home_team_history_opponent_goal_3', 'home_team_history_opponent_goal_4', 'home_team_history_opponent_goal_5', 'home_team_history_opponent_goal_6', 'home_team_history_opponent_goal_7', 'home_team_history_opponent_goal_8', 'away_team_history_opponent_goal_1', 'away_team_history_opponent_goal_2', 'away_team_history_opponent_goal_3', 'away_team_history_opponent_goal_4', 'away_team_history_opponent_goal_5', 'away_team_history

In [30]:
'is_cup' in the_best_features

False

In [ ]:
grouped_columns = []
for column in sorted(X_train.columns.values, key=lambda  x: x[-1]):
    if grouped_columns and column[:-3] in last_value:
        grouped_columns[-1].append(column)
    else:
        grouped_columns.append([column])
    last_value = column

model = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

the_best_features, the_best_score = features_selection(grouped_columns)
the_best_features